In [1]:

import pdb
import os
import traceback
import numpy as np
import jax
from jax import numpy as jnp
import optax
import wandb
import equinox as eqx
from typing import Optional, Callable
import librosa

from common_dl_utils.config_creation import Config
import common_jax_utils as cju

wandb.login()

key = jax.random.PRNGKey(12398)
key_gen = cju.key_generator(key)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: abdtab (abdtab-tue). Use `wandb login --relogin` to force relogin


In [2]:
def load_audio_file(file_path, sr=16000, save_npy=True):
    """
    Load an audio file and return it as a normalized numpy array.
    Optionally save as .npy file.
    """
    # Load the audio file
    audio, _ = librosa.load(file_path, sr=sr)
    
    # Convert to numpy array and normalize to [-1, 1] range
    audio = np.array(audio, dtype=np.float32)
    audio = audio / np.max(np.abs(audio))

    if save_npy:
        # Create npy filename from original audio filename
        npy_path = os.path.splitext(file_path)[0] + '.npy'
        np.save(npy_path, audio)
        return audio, len(audio), npy_path
   
    return audio, len(audio), None

In [3]:
# First set up the config properly
config = Config()

# Model architecture configuration
config.architecture = './model_components'  # module containing model classes
config.model_type = 'inr_modules.CombinedINR'

# Model configuration
config.model_config = Config()
config.model_config.in_size = 1  # Time dimension input
config.model_config.out_size = 1  # Audio amplitude output
config.model_config.terms = [
    ('inr_modules.MLPINR.from_config', {
        'hidden_size': 256,
        'num_layers': 5,
        'layer_type': 'inr_layers.FinerLayer',
        'num_splits': 1,
        'use_complex': False,
        'activation_kwargs': {'w0': 10},
        'initialization_scheme': 'initialization_schemes.finer_scheme',
        'initialization_scheme_kwargs': {'bias_k': 5}
    })
]

# Training configuration
config.trainer_module = './inr_utils'  # module containing training code
config.trainer_type = 'training.train_inr'

# Loss function configuration
config.loss_evaluator = config.loss_function = 'losses.SoundLossEvaluator'
config.loss_evaluator_config = config.loss_function_config = {
    'time_domain_weight': 1.0,
    'frequency_domain_weight': 0.1
}

# Optimizer configuration
config.optimizer = 'adam'  # will use optax.adam
config.optimizer_config = {
    'learning_rate': 1e-3
}

# Load and save the audio file
audio_path = './example_data/data_gt_bach.wav'  # Make sure this path exists
audio_data, fragment_length, npy_path = load_audio_file(audio_path)

# Sampler configuration
config.sampler = ('sampling.SoundSampler', {
    'window_size': 1024,
    'batch_size': 32,
    'fragment_length': fragment_length,  # Will be set after loading audio
    'sound_fragment': npy_path
})

# Callback configuration for logging
config.after_step_callback = 'callbacks.ComposedCallback'
config.after_step_callback_config = {
    'callbacks': [
        ('callbacks.print_loss', {'after_every': 10}),
        'callbacks.report_loss',
        'callbacks.raise_error_on_nan'
    ],
    'show_logs': False
}

#  ('callbacks.MetricCollectingCallback',
#             {'metric_collector': 'metrics.MetricCollector'}
#         ),
# # Metric collection configuration
# config.metric_collector_config = {
#     'metrics': [
#         ('metrics.MSEOnFixedGrid', {'grid': [1024], 'batch_size': 1024, 'frequency': 'every_n_batches'})
#     ],
#     'batch_frequency': 100,
#     'epoch_frequency': 1
# }

config.after_training_callback = None
config.optimizer_state = None
config.steps = 40000
config.use_wandb = True


In [4]:
try:
    inr = cju.run_utils.get_model_from_config_and_key(
        prng_key=next(key_gen),
        config=config,
        model_sub_config_name_base='model',
        add_model_module_to_architecture_default_module=False,
    )
except Exception as e:
    traceback.print_exc()
    print(e)
    print('\n')
    pdb.post_mortem()


In [5]:
# Then get the experiment
experiment = cju.run_utils.get_experiment_from_config_and_key(
    prng_key=next(key_gen),
    config=config,
    model_kwarg_in_trainer='inr',
    model_sub_config_name_base='model',
    trainer_default_module_key='trainer_module',
    additional_trainer_default_modules=[optax],
    add_model_module_to_architecture_default_module=False,
    initialize=False
)

In [6]:
# Run the experiment with wandb logging
with wandb.init(
    project='inr-audio',
    config={
        'window_size': config.sampler[1]['window_size'],
        'batch_size': config.sampler[1]['batch_size'],
        'learning_rate': config.optimizer_config['learning_rate'],
        'steps': config.steps,
        'audio_path': config.sampler[1]['sound_fragment']
    }
) as run:
    results = experiment.initialize()

Traceback (most recent call last):
  File "/tmp/ipykernel_15087/2184283701.py", line 12, in <module>
    results = experiment.initialize()
  File "/home/abdtab/miniconda3/envs/inr_edu_24/lib/python3.10/site-packages/common_dl_utils/config_realization.py", line 195, in initialize
    return cls(**processed_self_kwargs)
  File "/home/abdtab/INR_BEP/inr_utils/training.py", line 192, in train_inr
    inr, optimizer_state, loss, state = train_step(inr, optimizer_state, next(key_gen), state)
  File "/home/abdtab/miniconda3/envs/inr_edu_24/lib/python3.10/site-packages/equinox/_jit.py", line 242, in __call__
    return self._call(False, args, kwargs)
  File "/home/abdtab/miniconda3/envs/inr_edu_24/lib/python3.10/site-packages/equinox/_module.py", line 1078, in __call__
    return self.__func__(self.__self__, *args, **kwargs)
  File "/home/abdtab/miniconda3/envs/inr_edu_24/lib/python3.10/site-packages/equinox/_jit.py", line 215, in _call
    out = self._cached(dynamic_donate, dynamic_nodonate, 

TypeError: dot_general requires contracting dimensions to have the same shape, got (1,) and (1024,).